# Segmentação de Clientes com PyTorch (Buonopreço)
**Objetivo:** Evoluir da MLP “na mão” para uma **MLP em PyTorch** treinada com `TensorDataset` + `DataLoader` para **classificação multiclasse** (segmentação de clientes).

> **Você verá neste notebook:**
> - Carregamento e preparo do dataset **Buonopreço**.
> - Construção do alvo multiclasse (prioridade: `classe_cliente`; fallback por faixas de gasto).
> - Pré-processamento tabular com scikit-learn (numérico + categórico) e conversão para `torch.Tensor`.
> - MLP em PyTorch, treino com `DataLoader`, avaliação e curvas.
> - Salvamento e carregamento do modelo e do pipeline de pré-processamento.


In [ ]:
# Verificação de dependências principais
try:
    import torch, torch.nn as nn, torch.optim as optim
    from torch.utils.data import TensorDataset, DataLoader
    TORCH_OK = True
    print("PyTorch:", torch.__version__)
except Exception as e:
    TORCH_OK = False
    print("PyTorch não disponível neste ambiente. Execute localmente com PyTorch instalado.")

import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer, make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
import joblib

DATA_PATH = "dataset-buonopreco-registro_de_clientes.csv"

# Hiperparâmetros
SEED = 42
TEST_SIZE = 0.2
BATCH_SIZE = 32
HIDDEN = 32
EPOCHS = 100
LR = 0.03

np.random.seed(SEED)
if TORCH_OK:
    torch.manual_seed(SEED)



## 1) Carregar dataset e inspecionar


In [ ]:
df = pd.read_csv(DATA_PATH)
print("Dimensão:", df.shape)
print("Colunas:", list(df.columns)[:20], "...")
df.head(10)  # exibir preview



## 2) Construir o alvo multiclasse (segmentação)
Prioridade: usar **`classe_cliente`** se existir.  
Fallback (didático): criar 3 segmentos por **faixas de `gasto_medio`** (baixa/média/alta) usando tercis.


In [ ]:
def build_multiclass_target(df):
    df_ = df.copy()
    # Caso 1: usar 'classe_cliente' diretamente se existir (string/categoria)
    if "classe_cliente" in df_.columns:
        y_raw = df_["classe_cliente"].astype(str).fillna("desconhecido")
        X = df_.drop(columns=["classe_cliente"])
        note = "Alvo = 'classe_cliente' (multiclasse)."
        return X, y_raw, note

    # Caso 2: fallback por faixas de gasto_medio (3 classes)
    if "gasto_medio" in df_.columns:
        s = pd.to_numeric(df_["gasto_medio"], errors="coerce").fillna(0.0)
        # tercis -> rótulos
        q = s.quantile([0.33, 0.66]).values
        bins = [-np.inf, q[0], q[1], np.inf]
        labels = ["baixo_valor", "medio_valor", "alto_valor"]
        y_raw = pd.cut(s, bins=bins, labels=labels, include_lowest=True).astype(str)
        X = df_.drop(columns=["gasto_medio"])
        note = "Alvo criado por tercis de 'gasto_medio' (baixo/médio/alto)."
        return X, y_raw, note

    raise ValueError("Não encontrei 'classe_cliente' nem 'gasto_medio' para criar alvo multiclasse.")

X_raw, y_raw, target_note = build_multiclass_target(df)
print("Target note:", target_note)
print("Distribuição de classes:")
print(y_raw.value_counts())



## 3) Pré-processamento tabular (scikit-learn)
- Remoção de IDs.
- Numéricas: imputação (mediana) + padronização.
- Categóricas: imputação (mais frequente) + One-Hot (ignorando categorias desconhecidas).


In [ ]:
# Remover identificadores óbvios
id_like = [c for c in X_raw.columns if any(tok in c.lower() for tok in ("id", "codigo", "cpf"))]
if id_like:
    X_raw = X_raw.drop(columns=id_like)

# Separar treino/teste com estratificação
le = LabelEncoder()
y_enc = le.fit_transform(y_raw)  # 0..K-1
class_names = le.classes_.tolist()
print("Classes:", class_names)

X_train_df, X_test_df, y_train, y_test = train_test_split(
    X_raw, y_enc, test_size=TEST_SIZE, random_state=SEED, stratify=y_enc
)

# Column selectors
numeric_selector = selector(dtype_include=np.number)
categorical_selector = selector(dtype_exclude=np.number)

numeric_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])
categorical_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocess = ColumnTransformer(transformers=[
    ("num", numeric_pipe, numeric_selector),
    ("cat", categorical_pipe, categorical_selector)
], remainder="drop", verbose_feature_names_out=False)

# Fit no treino e transformar treino/teste
X_train_np = preprocess.fit_transform(X_train_df).astype(np.float32)
X_test_np  = preprocess.transform(X_test_df).astype(np.float32)

print("Shapes finais:", X_train_np.shape, X_test_np.shape)



## 4) TensorDataset + DataLoader


In [ ]:
if TORCH_OK:
    import torch
    from torch.utils.data import TensorDataset, DataLoader

    X_train_t = torch.from_numpy(X_train_np)
    y_train_t = torch.from_numpy(y_train.astype(np.int64))
    X_test_t  = torch.from_numpy(X_test_np)
    y_test_t  = torch.from_numpy(y_test.astype(np.int64))

    train_ds = TensorDataset(X_train_t, y_train_t)
    test_ds  = TensorDataset(X_test_t, y_test_t)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE)

    in_dim = X_train_np.shape[1]
    out_dim = len(class_names)
    in_dim, out_dim
else:
    print("PyTorch indisponível para criar DataLoader.")



## 5) Definir a MLP em PyTorch
Arquitetura simples: `Linear → ReLU → Linear` (saída = logits).  
Usaremos `CrossEntropyLoss` (já combina LogSoftmax + NLLLoss).


In [ ]:
if TORCH_OK:
    class MLPMulti(nn.Module):
        def __init__(self, in_features, hidden, out_features):
            super().__init__()
            self.net = nn.Sequential(
                nn.Linear(in_features, hidden),
                nn.ReLU(),
                nn.Linear(hidden, out_features)
            )
        def forward(self, x):
            return self.net(x)  # logits

    model = MLPMulti(in_features=in_dim, hidden=HIDDEN, out_features=out_dim)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=LR)
    print(model)
else:
    print("Modelo não criado (PyTorch indisponível).")



## 6) Treinamento
Fluxo: **forward → loss → backward → step** por época, usando mini-batches do `DataLoader`.


In [ ]:
import time

if TORCH_OK:
    def eval_accuracy(loader):
        model.eval()
        correct = total = 0
        with torch.no_grad():
            for xb, yb in loader:
                logits = model(xb)
                preds = logits.argmax(dim=1)
                correct += (preds == yb).sum().item()
                total += yb.size(0)
        return correct / total

    history = {"loss": [], "acc_tr": [], "acc_te": []}
    t0 = time.time()
    for epoch in range(1, EPOCHS+1):
        model.train()
        run_loss = 0.0
        for xb, yb in train_loader:
            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()
            run_loss += loss.item() * xb.size(0)
        epoch_loss = run_loss / len(train_loader.dataset)
        acc_tr = eval_accuracy(train_loader)
        acc_te = eval_accuracy(test_loader)
        history["loss"].append(epoch_loss)
        history["acc_tr"].append(acc_tr)
        history["acc_te"].append(acc_te)
        if epoch % max(1, EPOCHS//10) == 0 or epoch == 1:
            print(f"Época {epoch:3d}/{EPOCHS} | loss={epoch_loss:.4f} | acc_tr={acc_tr:.3f} | acc_te={acc_te:.3f}")
    t1 = time.time()
    print(f"Treino concluído em {t1 - t0:.2f}s")
else:
    print("Treino não executado (PyTorch indisponível).")



## 7) Curvas de treino


In [ ]:
if TORCH_OK and len(history["loss"])>0:
    plt.figure(figsize=(9,4))
    plt.plot(history["loss"], label="loss (train)")
    plt.xlabel("Época"); plt.ylabel("Loss"); plt.title("Curva de perda"); plt.grid(True); plt.legend()
    plt.show()

    plt.figure(figsize=(9,4))
    plt.plot(history["acc_tr"], label="Acurácia treino")
    plt.plot(history["acc_te"], label="Acurácia teste")
    plt.xlabel("Época"); plt.ylabel("Acurácia"); plt.title("Evolução da acurácia"); plt.grid(True); plt.legend()
    plt.show()
else:
    print("Sem histórico para plotes.")



## 8) Avaliação final (teste)
Relatório de classificação e matriz de confusão.


In [ ]:
if TORCH_OK:
    model.eval()
    with torch.no_grad():
        logits_te = model(X_test_t)
        y_pred = logits_te.argmax(dim=1).cpu().numpy()

    acc = accuracy_score(y_test, y_pred)
    print(f"Acurácia (teste): {acc:.3f}\n")
    print("Relatório de Classificação:")
    print(classification_report(y_test, y_pred, target_names=class_names))

    cm = confusion_matrix(y_test, y_pred)
    print("\nMatriz de Confusão:\n", cm)
else:
    print("Avaliação não executada (PyTorch indisponível).")



## 9) Salvar modelo e pipeline
Salvamos o `state_dict` do modelo, o `LabelEncoder` das classes e o `ColumnTransformer` usado no pré-processamento.


In [ ]:
MODEL_PATH = "buonopreco_mlp_multiclasse.pth"
PREP_PATH  = "buonopreco_preprocess.pkl"
LE_PATH    = "buonopreco_labelencoder.pkl"

if TORCH_OK:
    torch.save({"state_dict": model.state_dict(),
                "in_dim": in_dim,
                "hidden": HIDDEN,
                "out_dim": out_dim}, MODEL_PATH)
    joblib.dump(preprocess, PREP_PATH)
    joblib.dump(le, LE_PATH)
    print("Arquivos salvos:")
    print(MODEL_PATH)
    print(PREP_PATH)
    print(LE_PATH)
else:
    print("Artefatos não salvos (PyTorch indisponível).")



## 10) Carregar e usar o modelo (exemplo rápido)


In [ ]:
if TORCH_OK:
    ckpt = torch.load(MODEL_PATH, map_location="cpu")
    preprocess2 = joblib.load(PREP_PATH)
    le2 = joblib.load(LE_PATH)

    model2 = MLPMulti(ckpt["in_dim"], ckpt["hidden"], ckpt["out_dim"])
    model2.load_state_dict(ckpt["state_dict"])
    model2.eval()

    # Inferência de exemplo com as primeiras 5 linhas do conjunto de teste original (dataframe)
    X_test_df_sample = X_test_df.iloc[:5].copy()
    X_test_np2 = preprocess2.transform(X_test_df_sample).astype(np.float32)
    with torch.no_grad():
        logits = model2(torch.from_numpy(X_test_np2))
        pred_ids = logits.argmax(dim=1).numpy()
        preds = le2.inverse_transform(pred_ids)
    print("Predições (amostra de 5):", preds)
else:
    print("Recarregamento/uso não executado (PyTorch indisponível).")



## 11) Conclusão orientada ao negócio
A MLP multiclasse treinada com PyTorch mostrou capacidade de **diferenciar perfis de clientes** a partir de atributos cadastrais e históricos.  
Isso habilita ações como: **segmentação de campanhas**, **priorização de atendimento** e **alocação de benefícios** por grupo de valor.  
Com o modelo e o pré-processamento salvos, abre-se caminho para **reprodutibilidade**, **compartilhamento** e **implantação** em sistemas internos.
